# 1. Cài đặt thư viện

In [1]:
# Cài đặt thư viện transformers và torch
!pip install transformers torch

# 2. Bài 1: Khôi phục Masked Token (Masked Language Modeling)

## Sử dụng mô hình BERT (Encoder-only) để điền từ vào chỗ trống.

### 1. Tải pipeline "fill-mask"

In [3]:
from transformers import pipeline

# Mặc định pipeline này sẽ tải mô hình distilroberta-base (hoặc biến thể BERT tương tự)
mask_filler = pipeline("fill-mask")

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


### 2. Câu đầu vào với token [MASK]

In [7]:
# Lưu ý: Một số model dùng <mask> thay vì [MASK], nhưng pipeline thường tự xử lý
input_sentence = "Hanoi is the <mask> of Vietnam."
# Nếu code báo lỗi, hãy thử thay <mask> bằng [MASK] tùy theo model mặc định được tải về

input_sentence

'Hanoi is the <mask> of Vietnam.'

### 3. Thực hiện dự đoán (top_k=5)

In [6]:
predictions = mask_filler(input_sentence, top_k=5)

predictions

[{'score': 0.9341353178024292,
  'token': 812,
  'token_str': ' capital',
  'sequence': 'Hanoi is the capital of Vietnam.'},
 {'score': 0.02996085211634636,
  'token': 3497,
  'token_str': ' Republic',
  'sequence': 'Hanoi is the Republic of Vietnam.'},
 {'score': 0.010538393631577492,
  'token': 1867,
  'token_str': ' Capital',
  'sequence': 'Hanoi is the Capital of Vietnam.'},
 {'score': 0.005418903660029173,
  'token': 32357,
  'token_str': ' birthplace',
  'sequence': 'Hanoi is the birthplace of Vietnam.'},
 {'score': 0.0014444863190874457,
  'token': 1144,
  'token_str': ' heart',
  'sequence': 'Hanoi is the heart of Vietnam.'}]

### 4. In kết quả

In [9]:
print(f"Câu gốc: {input_sentence}")
print("-" * 30)
for pred in predictions:
    print(f"Từ dự đoán: '{pred['token_str']}' | Độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}")

Câu gốc: Hanoi is the <mask> of Vietnam.
------------------------------
Từ dự đoán: ' capital' | Độ tin cậy: 0.9341
 -> Câu hoàn chỉnh: Hanoi is the capital of Vietnam.
Từ dự đoán: ' Republic' | Độ tin cậy: 0.0300
 -> Câu hoàn chỉnh: Hanoi is the Republic of Vietnam.
Từ dự đoán: ' Capital' | Độ tin cậy: 0.0105
 -> Câu hoàn chỉnh: Hanoi is the Capital of Vietnam.
Từ dự đoán: ' birthplace' | Độ tin cậy: 0.0054
 -> Câu hoàn chỉnh: Hanoi is the birthplace of Vietnam.
Từ dự đoán: ' heart' | Độ tin cậy: 0.0014
 -> Câu hoàn chỉnh: Hanoi is the heart of Vietnam.


Trả lời câu hỏi Bài 1:

Mô hình có dự đoán đúng từ "capital" không?

=> Trả lời: Có (thường "capital" sẽ là dự đoán có độ tin cậy cao nhất).

Tại sao các mô hình Encoder-only như BERT lại phù hợp cho tác vụ này?

=> Trả lời: Vì kiến trúc Encoder có khả năng nhìn hai chiều (bidirectional). Để điền vào chỗ trống ở giữa câu ("Hanoi is the ... of Vietnam"), mô hình cần hiểu ngữ cảnh từ cả bên trái ("Hanoi") và bên phải ("Vietnam"). Encoder được thiết kế để làm chính xác việc này.

# Bài 2: Dự đoán từ tiếp theo (Next Token Prediction)

Sử dụng mô hình GPT (Decoder-only) để viết tiếp câu.

### Để kết quả có thể tái lập (tùy chọn), bạn có thể set seed

In [11]:
from transformers import pipeline, set_seed

set_seed(42)

### 1. Tải pipeline "text-generation" (thường dùng GPT-2 mặc định)

In [12]:
generator = pipeline("text-generation", model="gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


### 2. Đoạn văn bản mồi

In [13]:
prompt = "The best thing about learning NLP is"

prompt

'The best thing about learning NLP is'

### 3. Sinh văn bản

In [14]:
# max_length: độ dài tối đa
# num_return_sequences: số lượng câu muốn sinh ra
generated_texts = generator(prompt, max_length=50, num_return_sequences=1, truncation=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


### 4. In kết quả

In [15]:
print(f"Câu mồi: '{prompt}'")
print("-" * 30)
for i, text in enumerate(generated_texts):
    print(f"Văn bản sinh ra {i+1}:")
    print(text['generated_text'])

Câu mồi: 'The best thing about learning NLP is'
------------------------------
Văn bản sinh ra 1:
The best thing about learning NLP is that there are a few ways to pick up resources:

Learn something from others.

Learning from teachers.

Learning from friends.

Learning from friends who are knowledgeable and articulate.

Learning from family.

Learning from friends who have had a strong interest in learning NLP.

I wish I had found a good way to give back. I have done this, I have done this, I have done this, I have done this, I have done this, I have done this.

If you're looking for something new, or you look to learn something from other people, then this is the place to start. I encourage people to do this, and to share their experiences with others.

So when you're ready to start learning NLP, this is a great place to start.

What's next?

I'm always looking for the next big thing. I've been in the top 10 (in our top 10 list for NLP learning), my top 10 list for NLP learning, I'm

Trả lời câu hỏi Bài 2:

1, Kết quả sinh ra có hợp lý không?

=> Trả lời: Khá hợp lý về mặt ngữ pháp, tuy nhiên ý nghĩa có thể hơi ngẫu nhiên do bản chất của việc sinh văn bản tự động (Generative AI).

2, Tại sao các mô hình Decoder-only như GPT lại phù hợp cho tác vụ này?

=> Trả lời: Vì Decoder hoạt động theo cơ chế tự hồi quy (autoregressive) và nhìn một chiều (unidirectional). Nó được huấn luyện để dự đoán từ tiếp theo dựa trên chuỗi các từ đã xuất hiện trước đó, giống như cách con người viết văn bản tuần tự từ trái sang phải.

# 4. Bài 3: Tính toán Vector biểu diễn của câu (Sentence Representation)

## Lấy vector đặc trưng (embedding) của câu bằng phương pháp Mean Pooling từ mô hình BERT.

### 1. Chọn mô hình BERT base

In [16]:
import torch
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

### 2. Câu đầu vào

In [17]:
sentences = ["This is a sample sentence."]

sentences

['This is a sample sentence.']

### 3. Tokenize câu

In [20]:
# return_tensors='pt': trả về PyTorch tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

inputs

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 7099, 6251, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

### 4. Đưa qua mô hình để lấy hidden states

In [22]:
with torch.no_grad():
    outputs = model(**inputs)

# Lấy hidden state cuối cùng (batch_size, sequence_length, hidden_size)
last_hidden_state = outputs.last_hidden_state

last_hidden_state

tensor([[[-0.1993, -0.2101, -0.1950,  ..., -0.4733,  0.0861,  0.7103],
         [-0.5400, -0.7178, -0.2873,  ..., -0.7211,  0.5801,  0.3946],
         [-0.1421, -0.7375,  0.3737,  ..., -0.3740,  0.0750,  0.9687],
         ...,
         [ 0.1321, -0.2893, -0.0043,  ..., -0.1772, -0.2123, -0.1983],
         [ 0.4060,  0.0366, -0.7327,  ...,  0.4169, -0.3416, -0.4542],
         [ 0.0646, -0.2088, -0.1323,  ...,  0.5954, -1.0679,  0.0173]]])

### 5. Thực hiện Mean Pooling (Tính trung bình cộng có trọng số mask)

In [23]:
attention_mask = inputs['attention_mask']

attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1]])

In [24]:
# Mở rộng kích thước của mask để khớp với kích thước của hidden state
# Mask gốc: [Batch, Seq_len] -> Mở rộng: [Batch, Seq_len, Hidden_size]
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

In [25]:
# Nhân hidden state với mask (để triệt tiêu giá trị của các token padding về 0)
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)

In [26]:
# Tính tổng số token thực (không phải padding) để chia trung bình
# clamp(min=1e-9) để tránh lỗi chia cho 0
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)

In [27]:
# Tính trung bình
sentence_embedding = sum_embeddings / sum_mask

### 6. In kết quả

In [28]:
print("Vector biểu diễn của câu (5 giá trị đầu tiên):")
print(sentence_embedding[0][:5]) # Chỉ in 5 giá trị đầu để dễ nhìn
print("\nKích thước của vector:", sentence_embedding.shape)

Vector biểu diễn của câu (5 giá trị đầu tiên):
tensor([-0.0639, -0.4284, -0.0668, -0.3843, -0.0658])

Kích thước của vector: torch.Size([1, 768])


Trả lời câu hỏi Bài 3:

1, Kích thước (chiều) của vector biểu diễn là bao nhiêu? Con số này tương ứng với tham số nào?

=> Trả lời: Kích thước là (1, 768) (nếu batch size là 1). Chiều dài 768 tương ứng với tham số hidden_size (đôi khi gọi là d_model) của kiến trúc bert-base.

2, Tại sao chúng ta cần sử dụng attention_mask khi thực hiện Mean Pooling?

=> Trả lời: Để đảm bảo tính chính xác của phép tính trung bình.

+ Khi xử lý theo batch, các câu ngắn hơn được thêm các token đệm (padding tokens, thường là số 0) để bằng độ dài câu dài nhất.

+ Các token đệm này không mang ý nghĩa ngữ nghĩa.

+ Nếu tính trung bình cộng tất cả các token mà không dùng mask, các giá trị của token đệm sẽ làm "loãng" hoặc sai lệch vector biểu diễn của câu. attention_mask giúp mô hình bỏ qua (nhân với 0) các token đệm này.